In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()])
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()])
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:02<00:00, 4.56MB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 134kB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.26MB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.27MB/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
from logging import log
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu(x)
    return logits


model = Net().to(device)
print(model)

Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X,y) in enumerate(dataloader):
    X,y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred,y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0

  with torch.inference_mode():
    for X,y in dataloader:
      X,y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [ ]:
epochs = 5

for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300249 [    0/60000]
loss: 0.270104 [ 6400/60000]
loss: 0.189709 [12800/60000]
loss: 0.256206 [19200/60000]
loss: 0.154080 [25600/60000]
loss: 0.308158 [32000/60000]
loss: 0.132931 [38400/60000]
loss: 0.259164 [44800/60000]
loss: 0.289694 [51200/60000]
loss: 0.171225 [57600/60000]
Test Error: 
 Accuracy: 95.8%, Avg loss: 0.133384 

Epoch 2
-------------------------------
loss: 0.084424 [    0/60000]
loss: 0.077652 [ 6400/60000]
loss: 0.085432 [12800/60000]
loss: 0.052928 [19200/60000]
loss: 0.045256 [25600/60000]
loss: 0.109529 [32000/60000]
loss: 0.057815 [38400/60000]
loss: 0.100518 [44800/60000]
loss: 0.095929 [51200/60000]
loss: 0.075896 [57600/60000]
Test Error: 
 Accuracy: 97.0%, Avg loss: 0.096184 

Epoch 3
-------------------------------
loss: 0.036577 [    0/60000]
loss: 0.056785 [ 6400/60000]
loss: 0.040269 [12800/60000]
loss: 0.064657 [19200/60000]
loss: 0.036664 [25600/60000]
loss: 0.048727 [32000/60000]
loss: 0.034347 [38400/

In [ ]:
# Save the model
torch.save(model.state_dict(), 'mnist_model.pth')

print('Model trained and saved.')

Model trained and saved.


In [ ]:

# Load the saved model
model = Net().to(device)
model.load_state_dict(torch.load('mnist_model.pth', weights_only=True))
model.eval()

from PIL import Image

image_path = '2.png'
image = Image.open(image_path).convert('L')
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((28, 28))
])
image = transform(image).to(device)

# Make a prediction
with torch.no_grad():
    output = model(image)
    prediction = torch.argmax(output).item()

print(f'Predicted digit: {prediction}')

Predicted digit: 2
